---
title: "찾아바꾸기 기능 활용"
description: 찾아바꾸기를 활용한 파이썬 자동화 사례
output-file: find_replace.html
---

## 문제설정

문서 작업을 하면서 같은 의미지만 다르게 작성하여 형식을 통일하기 위해 문서를 처음부터 검토해야 하는 경우가 있습니다.
예를 들어 "2022년"이라고 쓰는 경우도 있고 "'22년"으로 적는 경우도 있습니다. 이를 모두 2022년으로 작성 방식을 통일하고자 한다면 찾아바꾸기를 통해 쉽게 달성할 수 있습니다.

만약 이런 바꿔야 하는 단어가 수십개가 된다면 어떻게 될까요?
붙여써야 하는 경우, 자주 틀리는 오탈자, 영문명으로 바로 작성하거나 이니셜로만 작성하는 등, 수십개의 케이스를 모두 적용하는 것은 상당히 귀찮고 오류가 발생하기 쉬운 일입니다.

이런 문제를 `hwpapi`를 사용해 해결해 보고자 합니다.

[국토부 보도자료](http://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?id=95086857)를 보면 임대차 시장 안정 및 3분기 부동산 정상화 방안이라는 문서를 볼 수 있습니다.

여기서 보면 '주거 안정'이라고 띄어 쓴 경우와 '주거안정'이라고 붙여쓴 경우가 있습니다.
![](img/주거안정.png)

유사하게 '분양가 상한제'와 같이 띄어 쓴 경우와 '분양가상한제'라고 붙여 쓴 경우가 있죠.
![](img/분양가상한제1.png)
![](img/분양가상한제2.png)

또한 '시범사업지'와 '시범 사업지'와 같이 경우에 따라 붙이거나 띄는 경우는 한국어 특성상 자주 발생합니다. 
![](img/시범사업지.png)

이런 항목을 모두 붙여 쓰는 스크립트를 짜보도록 하겠습니다.

해야 할 일은 

1. 문서 불러오기
2. 기존과 변경할 것 목록 만들기
3. 찾아 바꾸기

이렇게 3단계로 구성됩니다.

### 문서 불러오기

우선 패키지를 불러오고 문서를 불러 옵니다.
저는 `hwps/220621(안건_1,2)임대차_시장_안정_및_3분기_부동산_정상화_방안.hwp` 파일을 읽어 오겠습니다.

In [1]:
from hwpapi.core import App

app = App()
app.open("hwps/220621(안건_1,2)임대차_시장_안정_및_3분기_부동산_정상화_방안.hwp")

C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\functions.py:30: SyntaxWarning: invalid escape sequence '\s'
  m = re.search("(^.+?)\s[A-Z0-9]+\.HFT", text)
2025-09-25 10:59:53 - hwpapi.functions.get_hwp_objects - INFO - Found 0 running HWP objects


Rebuilding cache of generated files for COM support...
Checking 7D2B6F3C-1D95-4E0C-BF5A-5EE564186FBCx0x1x0
Could not add module (IID('{7D2B6F3C-1D95-4E0C-BF5A-5EE564186FBC}'), 0, 1, 0) - <class 'AttributeError'>: module 'win32com.gen_py.7D2B6F3C-1D95-4E0C-BF5A-5EE564186FBCx0x1x0' has no attribute 'CLSIDToClassMap'
Done.


2025-09-25 11:00:02 - hwpapi.functions.dispatch - WARNING - AttributeError occurred, clearing cache: module 'win32com.gen_py.7D2B6F3C-1D95-4E0C-BF5A-5EE564186FBCx0x1x0' has no attribute 'MinorVersion'
2025-09-25 11:00:05 - hwpapi.functions.dispatch - INFO - Successfully dispatched after cache clear: HWPFrame.HwpObject
2025-09-25 11:00:05 - hwpapi.core.Engine - INFO - Engine initialized successfully with CLSID: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-09-25 11:00:05 - hwpapi.core.App - INFO - Engine loaded successfully: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-09-25 11:00:05 - hwpapi.core.App - INFO - Registering DLL: C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\FilePathCheckerModuleExample.dll
2025-09-25 11:00:06 - hwpapi.core.App - INFO - App window visibility set to: True
2025-09-25 11:00:06 - hwpapi.core.App - INFO - App initialized successfully with all accessors


'c:/Users/freed/Documents/python_projects/007_hwpapi/nbs/01_tutorials/hwps/220621(안건_1,2)임대차_시장_안정_및_3분기_부동산_정상화_방안.hwp'

In [15]:
action = app.actions.RepeatFind
pset = action.pset
pset.find_charshape.bold = 1
pset.find_charshape.apply()
print(pset.find_charshape)

find_replace = app.api.HParameterSet.HFindReplace.FindCharShape

for key in dir(find_replace):
    print(key, getattr(find_replace, key))
find_replace

{   'name': 'CharShape',
    'values': {   'Offset': {   '기타': 0,
                                '기호': 0,
                                '사용자': 0,
                                '영어': 0,
                                '일어': 0,
                                '한글': 0,
                                '한자': 0},
                  'bold': True,
                  'border_fill': {   'backslash_flag': 0,
                                     'border_color_bottom': 0,
                                     'border_color_left': 0,
                                     'border_color_right': 0,
                                     'border_color_top': 0,
                                     'border_fill_3d': False,
                                     'border_type_bottom': 0,
                                     'border_type_left': 0,
                                     'border_type_right': 0,
                                     'border_type_top': 0,
                                     'border_w

<win32com.gen_py.HwpObject 1.0 Type Library.HCharShape instance at 0x1678084701744>

### 기존 단어와 변경할 단어 목록 만들기

아래와 같이 기존 단어와 변경할 단어를 만들어 둡니다.
여기서는 단순히 `list`를 사용했지만, `pandas` 등을 사용하면 엑셀 파일에서 관리할 수 있습니다.

In [2]:
words = [("분양가 상한제", "분양가상한제"), ("주거안정", "주거 안정"), ("시범사업지", "시범 사업지")]

### 찾아바꾸기

이렇게 까지 되면 나머지는 간단합니다. `words`를 순환 하면서 반복해 주기만 하면 됩니다.
모두 찾아바꾸기를 하면 어디를 바꾸었는지 확인하기 어렵기 때문에 바꾼 단어는 붉은 색으로 처리해서 쉽게 눈으로 확인해 볼 수 있게 하겠습니다.
그러기 위해서  `CharShape`이라고 하는 `dataclass`를 불러오겠습니다.

In [3]:
charshape = app.get_charshape()
charshape.text_color = "#0000FF"
charshape.bold = 1
print(charshape)
for old, new in words:
    app.replace_all(old, new, new_charshape=charshape)
app.get_charshape()

{   'name': 'CharShape',
    'values': {   'Offset': {   '기타': 0,
                                '기호': 0,
                                '사용자': 0,
                                '영어': 0,
                                '일어': 0,
                                '한글': 0,
                                '한자': 0},
                  'bold': True,
                  'border_fill': {   'backslash_flag': None,
                                     'border_color_bottom': 0,
                                     'border_color_left': 0,
                                     'border_color_right': 0,
                                     'border_color_top': 0,
                                     'border_fill_3d': None,
                                     'border_type_bottom': 0,
                                     'border_type_left': 0,
                                     'border_type_right': 0,
                                     'border_type_top': 0,
                                     'border

{   'name': 'CharShape',
    'values': {   'Offset': {   '기타': 0,
                                '기호': 0,
                                '사용자': 0,
                                '영어': 0,
                                '일어': 0,
                                '한글': 0,
                                '한자': 0},
                  'bold': True,
                  'border_fill': {   'backslash_flag': None,
                                     'border_color_bottom': 0,
                                     'border_color_left': 0,
                                     'border_color_right': 0,
                                     'border_color_top': 0,
                                     'border_fill_3d': None,
                                     'border_type_bottom': 0,
                                     'border_type_left': 0,
                                     'border_type_right': 0,
                                     'border_type_top': 0,
                                     'border

In [13]:
action = app.actions.CharShape
pset = action.pset
pset.text_color = "#123456"
pset.bold = True    
action.run()
app.get_charshape()
app.set_charshape(text_color="#671234")

{   'name': 'CharShape',
    'values': {   'Offset': {   '기타': 0,
                                '기호': 0,
                                '사용자': 0,
                                '영어': 0,
                                '일어': 0,
                                '한글': 0,
                                '한자': 0},
                  'bold': True,
                  'border_fill': {   'backslash_flag': None,
                                     'border_color_bottom': 0,
                                     'border_color_left': 0,
                                     'border_color_right': 0,
                                     'border_color_top': 0,
                                     'border_fill_3d': None,
                                     'border_type_bottom': 0,
                                     'border_type_left': 0,
                                     'border_type_right': 0,
                                     'border_type_top': 0,
                                     'border

In [17]:
charshape = app.get_charshape()
charshape.text_color = "#0000FF"
charshape.bold = 1
for old, new in words:
    app.actions.FindDlg()
    action = app.actions.AllReplace
    pset = action.pset
    print(old, new)
    pset.find_string = old
    pset.replace_string = new
    pset.apply()
    print(pset)
    print(app.api.HParameterSet.HFindReplace.FindString)
    print(app.api.HParameterSet.HFindReplace.ReplaceString)
    app.api.HParameterSet.HFindReplace.ReplaceCharShape.TextColor = 12322
    # pset.replace_charshape.text_color =  "#0000FF"
    # pset.replace_charshape
    action.run()

app.api.HParameterSet.HFindReplace.FindString, pset

분양가 상한제 분양가상한제
{   'name': 'FindReplace',
    'values': {   'all_word_forms': False,
                  'auto_spell': True,
                  'direction': 'down',
                  'find_charshape': {   'bold': False,
                                        'border_fill': {   'backslash_flag': 0,
                                                           'border_color_bottom': 0,
                                                           'border_color_left': 0,
                                                           'border_color_right': 0,
                                                           'border_color_top': 0,
                                                           'border_fill_3d': False,
                                                           'border_type_bottom': 0,
                                                           'border_type_left': 0,
                                                           'border_type_right': 0,
                                    

('',
 {   'name': 'FindReplace',
     'values': {   'all_word_forms': False,
                   'auto_spell': True,
                   'direction': 'down',
                   'find_charshape': {   'bold': False,
                                         'border_fill': {   'backslash_flag': 0,
                                                            'border_color_bottom': 0,
                                                            'border_color_left': 0,
                                                            'border_color_right': 0,
                                                            'border_color_top': 0,
                                                            'border_fill_3d': False,
                                                            'border_type_bottom': 0,
                                                            'border_type_left': 0,
                                                            'border_type_right': 0,
                                

코드를 실행하고 나면 아래와 같이 바뀐 단어는 붉은색으로 표시되게 됩니다.
![](img/주거안정_변경.png)
![](img/시범사업지_변경.png)

이렇게 변경된 사항을 눈으로 확인하고 최종적으로 단축키 등으로 정리하면 문서 전체적으로 맞춰야 하는 단어나 자주 틀리는 오탈자를 쉽게 관리 할 수 있게 됩니다.

In [6]:
charshape = app.get_charshape()
charshape.text_color = "#0000FF"
app.set_charshape(charshape)

{   'name': 'CharShape',
    'values': {   'Offset': {   '기타': 0,
                                '기호': 0,
                                '사용자': 0,
                                '영어': 0,
                                '일어': 0,
                                '한글': 0,
                                '한자': 0},
                  'bold': True,
                  'border_fill': {   'backslash_flag': None,
                                     'border_color_bottom': 0,
                                     'border_color_left': 0,
                                     'border_color_right': 0,
                                     'border_color_top': 0,
                                     'border_fill_3d': None,
                                     'border_type_bottom': 0,
                                     'border_type_left': 0,
                                     'border_type_right': 0,
                                     'border_type_top': 0,
                                     'border

In [7]:
app.move("DownOfCell")

False

In [8]:
dir(app.api)

['Application',
 'ArcType',
 'AutoNumType',
 'BorderShape',
 'BreakWordLatin',
 'BrushType',
 'CLSID',
 'Canonical',
 'CellApply',
 'CellShape',
 'CharShadowType',
 'CharShape',
 'CheckXObject',
 'Clear',
 'ColDefType',
 'ColLayoutType',
 'ConvertPUAHangulToUnicode',
 'CreateAction',
 'CreateField',
 'CreateID',
 'CreateMode',
 'CreatePageImage',
 'CreateSet',
 'CrookedSlash',
 'CurFieldState',
 'CurMetatagState',
 'CurSelectedCtrl',
 'DSMark',
 'DbfCodeType',
 'DeleteCtrl',
 'Delimiter',
 'DrawAspect',
 'DrawFillImage',
 'DrawShadowType',
 'EditMode',
 'Encrypt',
 'EndSize',
 'EndStyle',
 'EngineProperties',
 'ExportStyle',
 'FieldExist',
 'FileTranslate',
 'FillAreaType',
 'FindCtrl',
 'FindDir',
 'FindPrivateInfo',
 'FontType',
 'GetBinDataPath',
 'GetCurFieldName',
 'GetCurMetatagName',
 'GetFieldList',
 'GetFieldText',
 'GetFileInfo',
 'GetFontList',
 'GetHeadingString',
 'GetMessageBoxMode',
 'GetMetatagList',
 'GetMetatagNameText',
 'GetMousePos',
 'GetPageText',
 'GetPos',
 'Ge

In [9]:
app.api.KeyIndicator()

(True, 1, 1, 16, 1, 31, 44, 0, '')

In [10]:
app.api.KeyIndicator()

(True, 1, 1, 16, 1, 31, 44, 0, '')

In [11]:
dir(app.api.ParentCtrl)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']